Necessary Libraries

In [1]:
# pip install pdfplumber --user
# pip install requests --user
# pip install tika --user
# pip install BeautifulSoup --user
# pip install urllib --user
# pip install time --user

In [2]:
!python -V

Python 3.9.13


In [3]:
import csv
import numpy as np
import os
import os.path
import pandas as pd
import pdfplumber
import re
import requests
import sys
import tika
from bs4 import BeautifulSoup as bs
from datetime import datetime
from pathlib import Path
from urllib.request import urlopen
from tika import parser
from time import strftime, sleep

C:\Yeni klasör\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


This project has been developed for data scraping from monthly added PDF reports on the Mortgage Finance Forecast Archives website (https://www.mba.org/news-and-research/forecasts-and-commentary/mortgage-finance-forecast-archives). The data scraping framework developed in this project converts the tabular data in PDF files to ".csv" format. The developed data scraping framework integrates two different tasks such as web scraping and data extraction. The code uses three different function named as extract_table_columns_name, extract_pdf_data, and get_pdf_files. The get_pdf_files function downloads the pdf files from the archive, and then extracts tabular data and attribute names using extract_pdf_data and extract_table_columns_name functions. The entire structure of the data scraping framework checks out the archive manually for newly added reports and works as a fully automated manner. Besides, the regex expressions in the framework are also compatible with pattern formats that vary in different table structures.

In this project, two different packages used for data extraction from PDF files: pdfplumber and tika-python. At the stage of web scraping, the url links of the pdf files are parsed using lxml feature of the BeautifulSoup package.  

When the code is run for the first time, it creates two files named as archievefiles, archievecsvfiles and it also creates a ".csv" file named as archievelinks in cwd. When the code is run for the second time, it only downloads the pdf files in the newly added url links to the archive and performs other operations.

In [4]:
def extract_table_columns_name(text):
    text = text.replace('\n\n', '\n')
    text = text.rstrip('\r\n')
    eee = re.compile(r"^([\w,(+\- \d%]+[A-Za-z_$)]+) {1,4}([\d+]+.*)")
    eee2 = re.compile(r"^(?!([\w,(+\- \d%]+[A-Za-z_$)]+) {1,4}([\d+]+.*)).*|^[A-Z]+ [A-Za-z]+ \(\w+ \$\)")
    columns_subnames = []
    columns_upnames = []

    for line in text.split('\n'):
        if eee.match(line):
            line = re.search(eee,line)
            subnames = line.group(1)
            columns_subnames.append(subnames)
        else:
            columns_subnames.append('')
    for line in text.split('\n'):
        if eee2.match(line):
            line = re.search(eee2,line)
            upnames = line.group(0)
            columns_upnames.append(upnames)
        else:
            columns_upnames.append('')
    df1 = pd.DataFrame(columns_upnames)
    df1 = df1.replace('', np.nan).ffill(axis ='rows')
    df2 = pd.DataFrame(columns_subnames)
    index1 = (df2[0] == "")
    df1 = df1[index1==False]
    df2 = df2[df2[0] != ""]
    df3 = df1[0] + ':' + df2[0]
    index2 = df1[0] == df2[0]
    index_final = index1 | index2
    df3 = df3[index_final==False]
    time_information = pd.DataFrame(["file_date","Year","Quarter"])
    df_final = pd.concat([time_information, df3]).reset_index(drop = True)
    columns_name = df_final.values
    
    return columns_name

In [5]:
def extract_pdf_data(pattern4):
    with pdfplumber.open("./archievefiles/"+pattern4) as pdf:
        page = pdf.pages[0]
        text = page.extract_text()
    file_date_pattern = r"(\w+ \d+), (\d{4})"
    date_output = re.search(file_date_pattern, text).group()
    file_date = datetime.strptime(date_output, '%B %d, %Y').strftime('%Y-%m-%d')
    Year_pattern_1 = r"(\d{4} \d{4} \d{4})"
    Year_output_1 = re.search(Year_pattern_1, text).group()
    Year_output_1 = Year_output_1.split(" ")
    Year_output_1 = np.array(Year_output_1)
    Year_output_1 = np.repeat(Year_output_1, 4)
    Year_pattern_2 = r"(Q4\s)(20.*\b\d{4})"
    Year_output_2 = re.search(Year_pattern_2, text).groups()
    Year_output_2 = Year_output_2[1].split(" ")
    Year_output_2 = np.array(Year_output_2)
    Year = np.append(Year_output_1, Year_output_2)
    text_Quarter = text[text.rfind("MBA Mortgage Finance Forecast"):text.rfind("Housing Measures")]
    Quarter_pattern = r"([Q][1-4])"
    Quarter_output_1 = re.findall(Quarter_pattern, text_Quarter)
    Quarter_output_1 = np.array(Quarter_output_1)
    Quarter_output_2 = np.full(len(Year_output_2), "None")
    Quarter = np.append(Quarter_output_1, Quarter_output_2)
    parsed = parser.from_file("./archievefiles/"+pattern4)
    text = parsed['content']
    text = text[text.rfind("Housing Measures"):text.rfind("Notes")]
    text = ''.join(str(text).replace(',',''))
    text = re.sub(' +', ' ', text)
    eee = re.compile(r"^([\w,(+\- \d%]+[A-Za-z_$)]+) {1,4}([\d+]+.*)")
    line_items = []
    for line in text.split('\n'):
        if eee.match(line):
            line = re.search(eee,line)        
            values = line.group(2)
            if len(values.split(" "))>=12:
                line_items.append(values)
                new = np.asarray(line_items)
    columns_name = extract_table_columns_name(text)
    columns_name = columns_name.tolist()
    columns_name = np.reshape(columns_name, -1).tolist()
    new = np.asarray(new)
    df = pd.DataFrame(new)
    df = df[0].str.split(' ', expand=True)
    df = df.iloc[:, :-1]
    df = df.T
    column_date = np.repeat(file_date, len(df))
    arr = np.vstack((column_date, Year, Quarter))
    df2 = pd.DataFrame(arr)
    frames = [df2.T, df]  
    result = pd.concat(frames, axis=1, join='inner')
    result.columns=columns_name
    return result, columns_name

In [6]:
def get_pdf_files(url):
    try:
        urlopen(url)
        print("URL is valid")
        links = []
        html = urlopen(url).read()
        html_page = bs(html, features="lxml")
        og_url = html_page.find("meta",  property = "og:url")
        for link in html_page.find_all('a', href=True):
            current_link = link.get('href')
            last_under = current_link.rfind('?')
            current_link = current_link[:last_under]
            if current_link.endswith('pdf'):
                if og_url:
                    #print("currentLink:",current_link)
                    links.append("https://www.mba.org" + current_link)
    except IOError:
        print ("URL is invalid, please check out the URL!")
        sys.exit()
    print("All available pdf links:", links)
     
    if os.stat("archievelinks.csv").st_size == 0:
        df = pd.DataFrame(links).drop_duplicates(keep='first')
        current_link = df.values
        iteration = len(current_link)
    else:
        path_directory =  os.getcwd()
        archievelinks_data = pd.read_csv("archievelinks.csv", header=None)
        new_archievelinks_data = pd.DataFrame(links).drop_duplicates(keep='first')
        df_diff_archievelinks = pd.concat([new_archievelinks_data, archievelinks_data]).drop_duplicates(keep=False)
        if df_diff_archievelinks.empty:
            print('There is no newly added file !')
            return df_diff_archievelinks
        else:
            df = df_diff_archievelinks
            current_link = df.values
            iteration = len(current_link)
                
    for i in range(iteration):
        pattern1 = str(current_link[i]).rfind('source')+7
        pattern2 = str(current_link[i])[pattern1:]
        pattern3 = str(current_link[i]).rfind('forecast')+9
        pattern4 = str(current_link[i])[pattern3:-2]
        while True:
            r = requests.get(f"https://www.mba.org/docs/default-source/{pattern2}")
            if r.status_code == 200:
                completeName = os.path.join(directory_name, f"{pattern4}")            
                pattern_name = pattern4[0:-4]+".csv"
                completeName_csv = os.path.join(csv_directory_name, f"{pattern_name}")
                
                with open(completeName, 'wb') as f:
                    f.write(r.content)
                    print(f"File Downloaded and Saved As: {pattern4}")
                
                with open(completeName_csv, 'wb') as f1:
                    result, columns_name = extract_pdf_data(pattern4)
                    pattern_name = pattern4[0:-4]
                    result.to_csv(completeName_csv, index=False, header=columns_name) 
                    print(f"Table Data of {pattern4} is Extracted and Saved As: {pattern_name}.csv")
                break
            else:
                print("File Not Raised Yet, We Will Check Back After One Day.")
                sleep(3600)
                continue
    return df 

In [7]:
url = "https://www.mba.org/news-and-research/forecasts-and-commentary/mortgage-finance-forecast-archives"

directory_name = "archievefiles"
csv_directory_name = "archivecsvfiles"

if os.path.isfile('archievelinks.csv')==False and os.path.exists(directory_name)==False and os.path.exists(csv_directory_name)==False:
    path_directory =  os.getcwd()
    pdf_links = open('archievelinks.csv','a+')
    
    os.mkdir(os.path.join(path_directory, directory_name))
    Path('archievefiles').mkdir(parents=True, exist_ok=True)
    
    os.mkdir(os.path.join(path_directory, csv_directory_name))
    Path('archivecsvfiles').mkdir(parents=True, exist_ok=True)
    
    df = get_pdf_files(url)
    df = df.reindex(index=df.index[::-1])
    df.to_csv('archievelinks.csv', index=False, header=None)
    pdf_links.close()
else:
    pdf_links = open('archievelinks.csv','r')
    df = get_pdf_files(url)
    df = df.reindex(index=df.index[::-1])
    df.to_csv('archievelinks.csv', mode = "a", index=False, header=None)
    pdf_links.close()

URL is valid
All available pdf links: ['https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-aug-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-july-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-june-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-june-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-may-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-apr-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-mar-2022.pdf', 'https://www.mba.org/docs/default-source/research-and-forecasts/forecasts/mortgage-finance-forecast-feb-2022.pdf', 'https://www.mba.org/docs/default-sour

2022-09-10 23:19:45,424 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Table Data of aug-2022.pdf is Extracted and Saved As: aug-2022.csv
File Downloaded and Saved As: july-2022.pdf
Table Data of july-2022.pdf is Extracted and Saved As: july-2022.csv
File Downloaded and Saved As: june-2022.pdf
Table Data of june-2022.pdf is Extracted and Saved As: june-2022.csv
File Downloaded and Saved As: may-2022.pdf
Table Data of may-2022.pdf is Extracted and Saved As: may-2022.csv
File Downloaded and Saved As: apr-2022.pdf
Table Data of apr-2022.pdf is Extracted and Saved As: apr-2022.csv
File Downloaded and Saved As: mar-2022.pdf
Table Data of mar-2022.pdf is Extracted and Saved As: mar-2022.csv
File Downloaded and Saved As: feb-2022.pdf
Table Data of feb-2022.pdf is Extracted and Saved As: feb-2022.csv
File Downloaded and Saved As: jan_2022.pdf
Table Data of jan_2022.pdf is Extracted and Saved As: jan_2022.csv
File Downloaded and Saved As: dec_2021.pdf
Table Data of dec_2021.pdf is Extracted and Saved As: dec_2021.csv
File Downloaded and Saved As: nov-2021.pdf
Tabl